# Capital Asset Pricing Model (CAPM)
This notebook tests the single-factor CAPM for a set of assets by regressing excess asset returns on the market excess return.

**Source:** Essentials of Financial Economics  
**Authors:** Michael Donadelli, Michele Costola, Ivan Gufler  
**Date:** May 8, 2025


## 0. Preliminaries

This Python notebook estimates a CAPM model with a single risk factor, i.e. market risk.

We use returns of the 5 oldest stocks in the DJIA index: Procter and Gamble, 3M, Merkx, IBM and American Express, from February 2000 to December 2022. The risk-free rate is from Fama-French library and proxied by the one-month US Treasury bill. 


## 1. Import Libraries


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from IPython.display import display


## 2. Load and Prepare Data


In [3]:
Ret = pd.read_excel('../Data/Returns.xlsx')
Factors = pd.read_excel('../Data/FF_Factors.xlsx')

Mkt = np.array(Factors.iloc[1:, 1]) / 100
Rf = np.array(Factors.iloc[1:, 4]) / 100
ExRet = np.array(Ret.iloc[:, 1:]) - Rf.reshape(-1, 1)


## 3. Run CAPM Regressions
We regress each asset's excess return on the market excess return:

$$
R_{i,t} - R_f = \alpha_i + \beta_i (R_{M,t} - R_f) + \varepsilon_t.
$$


In [4]:
nS, mS = ExRet.shape

alpha = np.full((2, mS), np.nan)
beta = np.full((2, mS), np.nan)
R2 = np.full((1, mS), np.nan)

for i in range(mS):
    X = sm.add_constant(Mkt)
    y = ExRet[:, i]
    model = sm.OLS(y, X)
    results = model.fit()

    alpha[0, i] = round(results.params[0], 5)
    alpha[1, i] = round(results.tvalues[0], 5)
    beta[0, i] = round(results.params[1], 5)
    beta[1, i] = round(results.tvalues[1], 5)
    R2[0, i] = round(results.rsquared_adj, 5)


## 4. Display Results


In [5]:
Stocks = pd.DataFrame(
    data=np.concatenate((alpha, beta, R2)),
    columns=Ret.columns[1:],
    index=['alpha', '(alpha t-stat)', 'beta', '(beta t-stat)', 'Adj. R2']
)

print('CAPM Results:')
display(Stocks)



CAPM Results:


,PG,MMM,IBM,MRK,AXP
alpha,0.00499,0.00203,-0.00053,0.00333,0.00111
(alpha t-stat),1.64746,0.70967,-0.14983,0.83262,0.26176
beta,0.26819,0.79269,0.93822,0.44862,1.28948
(beta t-stat),4.12622,12.90035,12.33464,5.21768,14.16596
Adj. R2,0.05526,0.37645,0.35551,0.08735,0.42154


## 5. Save Results


In [11]:
Stocks.to_csv('CAPM_Stock.csv', index_label='Metric')
print("Results saved to 'CAPM_Stock.csv'.")


Results saved to 'CAPM_Stock.csv'.


## 6. Interpretation
- **Alpha (α)**: abnormal return not explained by market risk. α ≈ 0 is consistent with CAPM.
- **Beta (β)**: sensitivity to market movements (>1 more volatile, <1 less volatile than market).
- **Adjusted R²**: fraction of return variation explained by the market factor.
